<a href="https://colab.research.google.com/github/pikey-msc/RiesgosFinancieros/blob/master/2024-1/Insumos/BonoM_Cup%C3%B3nCero_c_trad_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython
!rm -rf RiesgosFinancieros
!git clone "https://github.com/pikey-msc/RiesgosFinancieros/"

Cloning into 'RiesgosFinancieros'...
remote: Enumerating objects: 903, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 903 (delta 84), reused 99 (delta 64), pack-reused 768
Receiving objects: 100% (903/903), 62.95 MiB | 18.70 MiB/s, done.
Resolving deltas: 100% (589/589), done.
Updating files: 100% (415/415), done.


In [ ]:
%%R
#Tasa alambrada
talamb=function(nodos,curva,plazos) #función de interpolación de tasas por el método alamabrada
{
  n=max(ncol(plazos),1)
  m=max(ncol(nodos),1)
  TC=matrix(0,1,n)
  TL=matrix(0,1,n)
  TF=matrix(0,1,n)
  for (j in 1:n)
  {
    i=1
    repeat
    {
      if(nodos[i]<= plazos[j] && plazos[j] <=nodos[i+1])
      {
        TC[j]=curva[i]
        TL[j]=curva[i+1]
        TF[j]=((((1+TL[j]*nodos[i+1]/360)/(1+TC[j]*nodos[i]/360))^((plazos[j]-nodos[i])/(nodos[i+1]-nodos[i]))*(1+TC[j]*nodos[i]/360))-1)*360/plazos[j]
        break
      }
      else if (plazos[j]<nodos[1])
      {
        TC[j]=curva[1]
        TL[j]=curva[1]
        TF[j]=curva[1]
        break
      }
      else if (plazos[j]>nodos[m])
      {
        TC[j]=curva[m]
        TL[j]=curva[m]
        TF[j]=curva[m]
        break
      }
      else
      {i=i+1}
    }
  }
  as.matrix(t(as.numeric(rbind(TF))))
}

In [ ]:
%%R
btasadesc="RiesgosFinancieros/2020-1/Insumos/tasa_guber.txt"
tfcupon=cbind( 0.065,	0.0675,	0.07,	0.075,	0.078) #Tasafija del cupón
plazos=cbind( 378,	405,	550,	1200,	1800) #Vencimiento del bono
plazocupon=cbind( 182,	182, 182, 182, 182) #plazos fijos de cada cupón
contratos=cbind(22000, -29000, 29000, -46000, 10000) #posición invertida
nominal=100
itpl=0 #poner 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada

In [ ]:
%%R
#carga de datos
#carga de tasas de descuento
data1<-read.table(btasadesc)
n<-nrow(data1)
m_orig=ncol(data1)
X_orig=data.frame(data1[2:n,2:m_orig])
nodos=data.frame(data1[1,2:m_orig])
n=n-1
#Posición inicial
#interpolación de tasas y volatilidades
m=ncol(plazos)


      N=as.integer(plazos/plazocupon)+1 #número de cupones a pagar
      VTplazos=matrix(0,1,sum(N)) #vector de todos los plazos de todos los contratos
      contratosT=matrix(0,1,sum(N)) #vector de todos los contratos de todos los flujos de todos los contratos
      nominalT=matrix(0,1,sum(N)) #vector de todos los nominales de todos los flujos de todos los contratos
      plazocuponT=matrix(0,1,sum(N)) #vector de todos los plazoscupon de todos los flujos de todos los contratos
      tasafijaT=matrix(0,1,sum(N)) #vector de tasas fijas de todos los flujos de todos los contratos
      ulNomT=matrix(0,1,sum(N)) #vector de contratos a final de flujo


      plazini=plazos-plazocupon*(N-1) #vector de plazos iniciales

      for (j in (1:m))
      {
        if (j==1)
        {
          VTplazos[,1:sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
          contratosT[,1:sum(N[1:j])]=seq(contratos[j],contratos[j])
          plazocuponT[,1:sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
          tasafijaT[,1:sum(N[1:j])]=seq(tfcupon[j],tfcupon[j])
          ulNomT[,sum(N[1:j])]=contratos[j]
        }
        else
        {
          VTplazos[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazini[j],plazos[j], by=plazocupon[j])
          contratosT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(contratos[j],contratos[j])
          plazocuponT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(plazocupon[j],plazocupon[j])
          tasafijaT[,(sum(N[1:j-1])+1):sum(N[1:j])]=seq(tfcupon[j],tfcupon[j])
          ulNomT[,sum(N[1:j])]=contratos[j]
        }
      }

      Xvp=matrix(0,n,ncol(VTplazos))

      for (i in (1:n))
      {
        Xvp[i,]=if(itpl==0){approx(nodos,X_orig[i,],VTplazos)$y}else{talamb(nodos,X_orig[i,],VTplazos)}

      }

      print(VTplazos)
      ulNomT
      print(tasafijaT)
      N

     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11] [,12] [,13] [,14]
[1,]   14  196  378   41  223  405    4  186  368   550   108   290   472   654
     [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24] [,25] [,26]
[1,]   836  1018  1200   162   344   526   708   890  1072  1254  1436  1618
     [,27]
[1,]  1800
      [,1]  [,2]  [,3]   [,4]   [,5]   [,6] [,7] [,8] [,9] [,10] [,11] [,12]
[1,] 0.065 0.065 0.065 0.0675 0.0675 0.0675 0.07 0.07 0.07  0.07 0.075 0.075
     [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20] [,21] [,22] [,23] [,24]
[1,] 0.075 0.075 0.075 0.075 0.075 0.078 0.078 0.078 0.078 0.078 0.078 0.078
     [,25] [,26] [,27]
[1,] 0.078 0.078 0.078
[1]  3  3  4  7 10


$$ V_{cc}=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$

Donde
$V_{cc}$: Valor del Bono bajo tasa de curva cupón cero (Precio Sucio)
$N$: Valor Nominal del bono.
$C$: Número de contratos.
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.
$p_{i}$: Plazo acumulado (en días) al cupón $i$.
$t_{c}$: Tasa cupón fija
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos cupón cero según el plazo acumulado al pago del cupón $i$
Se tiene nun factor de riesgo subyacente (curva gubernamental) y $n$ factores de riesgos.


In [ ]:
%%R
# library(plyr)
bonoMcccero=function(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, N)
{
        V0=matrix(0,1,nrow(as.matrix(N)))
          V0f=((((contratosT*(tasafijaT)*(plazocuponT/360))+ulNomT)/(1+Xvp*VTplazos/360)))*nominal
        for (j in (1:nrow(as.matrix(N))))
        {
          if(j==1)
          {
            V0[j]=sum(V0f[j:N[j]])
          }
          else
          {
            V0[j]=sum(V0f[(sum(N[1:j-1])+1):(sum(N[1:j]))])
          }
        }
        V0
}


V0=bonoMcccero(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp[1,], N)
V0

        [,1]     [,2]    [,3]     [,4]    [,5]
[1,] 2236227 -2941747 2968292 -4643548 1009418


##TRADUCCIÓN A PYTHON

In [ ]:
#celda 1
import numpy as np

def talamb(nodos, curva, plazos):
    n = max(plazos.shape[1], 1)
    m = max(nodos.shape[1], 1)
    TC = np.zeros((1, n))
    TL = np.zeros((1, n))
    TF = np.zeros((1, n))

    for j in range(n):
        i = 0
        while i < m - 1:
            if nodos[0, i] <= plazos[0, j] <= nodos[0, i + 1]:
                TC[0, j] = curva[0, i]
                TL[0, j] = curva[0, i + 1]
                TF[0, j] = ((((1 + TL[0, j] * nodos[0, i + 1] / 360) / (1 + TC[0, j] * nodos[0, i] / 360)) **
                            ((plazos[0, j] - nodos[0, i]) / (nodos[0, i + 1] - nodos[0, i])) *
                            (1 + TC[0, j] * nodos[0, i] / 360)) - 1) * 360 / plazos[0, j]
                break
            elif plazos[0, j] < nodos[0, 0]:
                TC[0, j] = curva[0, 0]
                TL[0, j] = curva[0, 0]
                TF[0, j] = curva[0, 0]
                break
            elif plazos[0, j] > nodos[0, m - 1]:
                TC[0, j] = curva[0, m - 1]
                TL[0, j] = curva[0, m - 1]
                TF[0, j] = curva[0, m - 1]
                break
            else:
                i += 1

    return TF




In [ ]:
#celda 2
import numpy as np

# Ruta del archivo de tasas
btasadesc = "RiesgosFinancieros/2020-1/Insumos/tasa_guber.txt"

# Tasas fijas del cupón
tfcupon = np.array([0.065, 0.0675, 0.07, 0.075, 0.078])

# Vencimiento del bono
plazos = np.array([378, 405, 550, 1200, 1800])

# Plazos fijos de cada cupón
plazocupon = np.array([182, 182, 182, 182, 182])

# Posición invertida
contratos = np.array([22000, -29000, 29000, -46000, 10000])

# Nominal
nominal = 100

# Tipo de interpolación (0 para interpolación lineal, 1 para tasa alambrada)
itpl = 1


In [ ]:
#celda 3
import numpy as np
import pandas as pd

# Carga de datos - supongamos que los datos se encuentran en un archivo CSV llamado "btasadesc.csv"
# Asegúrate de proporcionar la ruta completa al archivo o que el archivo esté en el directorio actual.
# data1 = pd.read_("btasadesc.txt", header=None)
data1 = pd.read_csv("RiesgosFinancieros/2020-1/Insumos/tasa_guber.txt", sep="\t", header=None)

# Obtiene el número de filas y columnas de data1
n, m_orig= data1.shape

# Crear un DataFrame con los datos sin la primera fila y la primera columna
X_orig = np.matrix(pd.DataFrame(data1.values[1:, 1:len(data1.values[0])-1], dtype=float))

# Obtener nodos
nodos = pd.DataFrame(data1.values[0, 1:(len(data1.values[0])-1)], dtype=int)
n = n - 1

# Define las variables plazos, plazocupon, tfcupon y itpl que faltan en tu código
plazos = np.array([378, 405, 550, 1200, 1800])
plazocupon = np.array([182, 182, 182, 182, 182])
tfcupon = np.array([0.065, 0.0675, 0.07, 0.075, 0.078])
itpl = 0  # Pon 0 si se quiere interpolación lineal o 1 si se quiere tasa alambrada

m=len(plazos)


In [ ]:
import numpy as np

# Número de cupones a pagar
N = (plazos / plazocupon).astype(int) + 1

# Vector de todos los plazos de todos los contratos
VTplazos = np.zeros((1, int(np.sum(N))))

# Vector de todos los contratos de todos los flujos de todos los contratos
contratosT = np.zeros((1, int(np.sum(N))))

# Vector de todos los nominales de todos los flujos de todos los contratos
nominalT = np.zeros((1, int(np.sum(N))))

# Vector de todos los plazos de cupón de todos los flujos de todos los contratos
plazocuponT = np.zeros((1, int(np.sum(N))))

# Vector de tasas fijas de todos los flujos de todos los contratos
tasafijaT = np.zeros((1, int(np.sum(N))))

# Vector de contratos a final de flujo
ulNomT = np.zeros((1, int(np.sum(N))))

# Vector de plazos iniciales
plazini = plazos - plazocupon * (N - 1)

for j in range(m):
  # print(j)
  if j == 0:
      VTplazos[:, 0:int(np.sum(N[0:j+1]))] = np.arange(plazini[j], plazos[j] + plazocupon[j], plazocupon[j])
      contratosT[:, 0:int(np.sum(N[0:j+1]))] = np.arange(contratos[j], contratos[j] + 1)
      plazocuponT[:, 0:int(np.sum(N[0:j+1]))] = plazocupon[j] * np.ones(int(np.sum(N[0:j+1])))
      tasafijaT[:, 0:int(np.sum(N[0:j+1]))] = tfcupon[j] * np.ones(int(np.sum(N[0:j+1])))
      ulNomT[:, N[j]-1] = contratos[j]
  else:
      VTplazos[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = np.arange(plazini[j], plazos[j] + plazocupon[j], plazocupon[j])
      contratosT[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = np.arange(contratos[j], contratos[j] + 1)
      plazocuponT[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = plazocupon[j] * np.ones(int(np.sum(N[j:j+1])))
      tasafijaT[:, int(np.sum(N[0:j])):int(np.sum(N[0:j+1]))] = tfcupon[j] * np.ones(int(np.sum(N[j:j+1])))
      ulNomT[:, int(np.sum(N[0:j+1]))-1] = contratos[j]

Xvp = np.zeros((n, VTplazos.shape[1]))

for i in range(n):
    if itpl == 0:
        # Utiliza interpolación lineal
        Xvp[i, :] = np.interp(VTplazos[0, :],np.array(nodos[0]), np.array(X_orig[i])[0])
    else:
        # Utiliza la función talamb
        # (Asegúrate de que tengas definida la función talamb)
        Xvp[i, :] = talamb(nodos, X_orig[i, :], VTplazos[0, :])

print(VTplazos)
print(ulNomT)
print(contratosT)
print(tasafijaT)
print(np.array(X_orig[i][0]))
# print(N)
print(N[0])
ulNomT[:, N[0]]
contratos[0]

[[  14.  196.  378.   41.  223.  405.    4.  186.  368.  550.  108.  290.
   472.  654.  836. 1018. 1200.  162.  344.  526.  708.  890. 1072. 1254.
  1436. 1618. 1800.]]
[[     0.      0.  22000.      0.      0. -29000.      0.      0.      0.
   29000.      0.      0.      0.      0.      0.      0. -46000.      0.
       0.      0.      0.      0.      0.      0.      0.      0.  10000.]]
[[ 22000.  22000.  22000. -29000. -29000. -29000.  29000.  29000.  29000.
   29000. -46000. -46000. -46000. -46000. -46000. -46000. -46000.  10000.
   10000.  10000.  10000.  10000.  10000.  10000.  10000.  10000.  10000.]]
[[0.065  0.065  0.065  0.0675 0.0675 0.0675 0.07   0.07   0.07   0.07
  0.075  0.075  0.075  0.075  0.075  0.075  0.075  0.078  0.078  0.078
  0.078  0.078  0.078  0.078  0.078  0.078  0.078 ]]
[[0.07906    0.07671229 0.07142146 0.07607202 0.08243597 0.08497519
  0.08559156 0.08727407 0.09459063 0.10000798 0.10205611 0.10166534
  0.10144164 0.10201717 0.10292152 0.10526908 0.1204

22000

# Ingresar la ecuación al chat (que lo haga para valuar varios bonos en una función):
$$ V_{cc}=\sum_{i=1}^{n}\frac{N\cdot C \cdot t_{c}\cdot p_c/360}{(1+t_{vp_{p_i}} \cdot p_i/360)} + \frac{N\cdot C}{(1+t_{vp_{p_n}} \cdot p_n/360)}$$

Donde
$V_{cc}$: Valor del Bono bajo tasa de curva cupón cero (Precio Sucio)\
$N$: Valor Nominal del bono.\
$C$: Número de contratos.\
$p_{c}$: Plazo fijo para cada pago de intereses del cupón.\
$p_{i}$: Plazo acumulado (en días) al cupón $i$.\
$t_{c}$: Tasa cupón fija\
$t_{vp_{p_i}}$: Tasa valor presente que depende de la curva de bonos cupón cero según el plazo acumulado al pago del cupón $i$\
Se tiene nun factor de riesgo subyacente (curva gubernamental) y $n$ factores de riesgos.


In [ ]:
import numpy as np

def bonoMcccero(contratosT, nominal, tasafijaT, plazocuponT, VTplazos, Xvp, N):
    V0 = np.zeros((1, len(N)))[0]
    V0f = ((((contratosT * tasafijaT * (plazocuponT / 360)) + ulNomT) / (1 + Xvp * VTplazos / 360)) * nominal)[0]
    for j in range(len(N)):
        if j == 0:
            V0[j] = np.sum(V0f[j:N[j]])
        else:
            V0[j] = np.sum(V0f[sum(N[0:j]):sum(N[0:j+1])])
    return V0

V0 = bonoMcccero(contratosT[0], nominal, tasafijaT[0], plazocuponT[0], VTplazos[0], Xvp[0], N)
print(V0)


[ 2236227.49404477 -2941747.37154949  2968292.0331245  -4643547.65817406
  1009417.93928807]
